In [3]:
import copy

import pandas as pd
import numpy as np
import os

In [4]:
# 添加验证集和测试集
train_df = pd.read_csv('../UJIIndoorLoc/trainingData.csv')
train_df_noisy = pd.read_csv('../data/train_noisy.csv')
train_df_noisy1 = pd.read_csv('../data/train_noisy1.csv')
# train_df_noisy2 = pd.read_csv('../data/train_noisy2.csv')
# train_df_noisy3 = pd.read_csv('../data/train_noisy3.csv')
# train_df_noisy4 = pd.read_csv('../data/train_noisy4.csv')
train_df_noisy5 = pd.read_csv('../data/train_noisy5.csv')

# train_df = pd.concat([train_df, train_df_noisy4], ignore_index=True)
# train_df = pd.concat([train_df, train_df_noisy1], ignore_index=True)
# train_df = pd.concat([train_df, train_df_noisy2], ignore_index=True)
# train_df = pd.concat([train_df, train_df_noisy3], ignore_index=True)
# train_df = pd.concat([train_df, train_df_noisy5], ignore_index=True)
valid_df = pd.read_csv('../UJIIndoorLoc/validationData.csv')

In [5]:
# 训练集特征
training_data = train_df[train_df.columns[:520]].to_numpy()
training_floors = train_df['FLOOR'].to_numpy() # FLOOR LABELS
training_buildings = train_df['BUILDINGID'].to_numpy() # BUILDING LABELS
training_longitude = train_df['LONGITUDE'].to_numpy() # LONGITUDE LABELS
training_latitude = train_df['LATITUDE'].to_numpy() # LATITUDE LABELS
training_coords = train_df[['LONGITUDE','LATITUDE']].to_numpy() # LONGITUDE + LATITUDE LABELS

In [6]:
# 验证机特征
valid_data = valid_df[valid_df.columns[:520]].to_numpy()
valid_floors = valid_df['FLOOR'].to_numpy() # FLOOR LABELS
valid_buildings = valid_df['BUILDINGID'].to_numpy() # BUILDING LABELS
valid_longitude = valid_df['LONGITUDE'].to_numpy() # LONGITUDE LABELS
valid_latitude = valid_df['LATITUDE'].to_numpy() # LATITUDE LABELS
valid_coords = valid_df[['LONGITUDE','LATITUDE']].to_numpy() # LONGITUDE + LATITUDE LABELS

In [7]:
# Augmenting data shape to fit 23x23
training_data_aug = np.empty((len(training_data),529))
for x in range(len(training_data)):
    training_data_aug[x] = np.concatenate((training_data[x], np.full(shape=9,fill_value=100)))
training_data_aug.shape

(19937, 529)

In [8]:
from sklearn.model_selection import train_test_split
X_trainCo_cnn, X_testCo_cnn, y_trainCo_cnn, y_testCo_cnn = train_test_split(training_data_aug.reshape(len(training_data_aug),23,23), training_coords, test_size=0.2, random_state=2812)

In [9]:
print(X_trainCo_cnn[:5])
print(X_testCo_cnn[:5])
print(y_trainCo_cnn[:5])
print(y_testCo_cnn[:5])


[[[100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. -92.]
  ...
  [100. 100. 100. ... 100. 100. 100.]
  [-78. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]]

 [[100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  ...
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]]

 [[100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... -88. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  ...
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]]

 [[100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... -50. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  ...
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. ... 100. 100. 100.]]

 [[100. 100. 100. ... 100. 100. 100.]
  [100. 100. 100. 

In [10]:
from utill.data_standar import normalize_rssi, normalize_coords
X_trainCo_cnn, X_trainCo_rssi_min, X_trainCo_rssi_max = normalize_rssi(X_trainCo_cnn)
X_testCo_cnn, X_testCo_rssi_min, X_testCo_rssi_max = normalize_rssi(X_testCo_cnn)
y_trainCo_cnn, y_trainCo_rssi_min, y_trainCo_rssi_max = normalize_coords(y_trainCo_cnn)
y_testCo_cnn, y_testCo_rssi_min, y_testCo_rssi_max = normalize_coords(y_testCo_cnn)
min_max_dist = [y_trainCo_rssi_min, y_trainCo_rssi_max, y_testCo_rssi_min, y_testCo_rssi_max]

TypeError: normalize_coords() missing 1 required positional argument: 'coords'

In [ ]:
print(y_trainCo_cnn[:5])
print(y_testCo_cnn[:5])

In [ ]:
import matplotlib.pyplot as plt
single_image = X_trainCo_cnn[0]
plt.imshow(single_image)
plt.title(f'Label: {y_trainCo_cnn[0]}')
plt.colorbar()
plt.show()

In [ ]:
from CNN import CNNRegressor
from utill.data_standar import denormalize_coords, mean_euclidean_distance
# Longitude+Latitude Regressor
dropout = [0.1]
val_kernel = [7]
val_padding = [3]
for val in dropout:
    for i in range(1):
        print(f'(Dropout: {val}) (kernel: {val_kernel[i]}) (padding: {val_padding[i]}) (lr: 0.001) (batch_size=100)')
        cnn = CNNRegressor(n_targets=2, dropout=val, kernel_size=val_kernel[i], padding=val_padding[i]) # Longitude + Latitude are targets
        cnn.fit(X_trainCo_cnn, y_trainCo_cnn, X_testCo_cnn, y_testCo_cnn)
        # cnn.fit(X_trainCo_cnn, y_trainCo_cnn_scaled, X_testCo_cnn, y_testCo_cnn_scaled)
        # cnn.train(num_epochs=100,eval_train=True, min_max_dist=min_max_dist, lr=0.0001, batch_size=16)
        cnn.train(num_epochs=100,eval_train=True, min_max_dist=min_max_dist, lr=0.0001, batch_size=64)
